In [ ]:
!pip install langchain_community
!pip install langchain_chroma
!pip install langchain_huggingface

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import pandas as pd

In [ ]:
books = pd.read_csv("books_clean.csv")

In [ ]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [ ]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep = "\n",
                                   index = False,
                                   header = False)

In [ ]:
raw_documents = TextLoader("tagged_description.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="Qwen/Qwen3-Embedding-0.6B",
    encode_kwargs={
        'batch_size': 2,  # Reduce from default 32, so Colab GPU can handle
        'normalize_embeddings': True
    }
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


In [ ]:
# Create and save database
db_books = Chroma.from_documents(
    documents,
    persist_directory="./chroma_db",
    embedding=embeddings
)

# Load existing database
# db_books = Chroma(
#     persist_directory="./chroma_db",
#     embedding_function=embeddings
# )

In [ ]:
# Zip folder so you can download to local storage
!zip -r chroma_db.zip chroma_db/

# Unzip folder to load database
# !unzip chroma_db.zip
# %ls chroma_db

  adding: chroma_db/ (stored 0%)
  adding: chroma_db/chroma.sqlite3 (deflated 34%)
  adding: chroma_db/d00ba544-f4b9-4da6-810f-f9212de24f18/ (stored 0%)
  adding: chroma_db/d00ba544-f4b9-4da6-810f-f9212de24f18/length.bin (deflated 90%)
  adding: chroma_db/d00ba544-f4b9-4da6-810f-f9212de24f18/data_level0.bin (deflated 8%)
  adding: chroma_db/d00ba544-f4b9-4da6-810f-f9212de24f18/link_lists.bin (deflated 75%)
  adding: chroma_db/d00ba544-f4b9-4da6-810f-f9212de24f18/header.bin (deflated 55%)
  adding: chroma_db/d00ba544-f4b9-4da6-810f-f9212de24f18/index_metadata.pickle (deflated 45%)


In [ ]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(id='011a23e5-d586-48c1-8aa7-ead15da24b7a', metadata={'source': 'tagged_description.txt'}, page_content="9780786808373 Introducing your baby to birds, cats, dogs, and babies through fine art, illsutration and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing baby to some basic -- and sometimes playful -- information on the subjects."),
 Document(id='e2c7cad6-c3ae-4187-b61b-c003d4c747f9', metadata={'source': 'tagged_description.txt'}, page_content="9780786808373 Introducing your baby to birds, cats, dogs, and babies through fine art, illsutration and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing baby to some basic -- and sometim

In [ ]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]

In [ ]:
retrieve_semantic_recommendations("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
324,9780060959036,0060959037,Prodigal Summer,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,Prodigal Summer: A Novel,9780060959036 Barbara Kingsolver's fifth novel...
816,9780142402931,0142402931,The Far Side of Evil,Sylvia Engdahl,Juvenile Fiction,http://books.google.com/books/content?id=7nijj...,A young girl from an advanced civilization is ...,2005.0,3.98,324.0,57.0,The Far Side of Evil,9780142402931 A young girl from an advanced ci...
855,9780143037392,0143037390,The Read-aloud Handbook,Jim Trelease,Language Arts & Disciplines,http://books.google.com/books/content?id=B2_yU...,Explains the importance of reading aloud to ch...,2006.0,4.40,432.0,4122.0,The Read-aloud Handbook,9780143037392 Explains the importance of readi...
1639,9780374422080,0374422087,Everything on a Waffle,Polly Horvath,Juvenile Fiction,http://books.google.com/books/content?id=NimVJ...,This Newbery Honor Book tells the story of 11 ...,2004.0,3.71,150.0,9631.0,Everything on a Waffle,9780374422080 This Newbery Honor Book tells th...
1739,9780375812934,0375812938,Jurassic Park Institute Dinosaur Field Guide,Thomas R. Holtz;M. K. Brett-Surman,Juvenile Nonfiction,http://books.google.com/books/content?id=vW3hm...,Discusses dinosaur research and what can be le...,2001.0,4.08,160.0,2.0,Jurassic Park Institute Dinosaur Field Guide,9780375812934 Discusses dinosaur research and ...
1907,9780393315110,0393315118,Uncommon Ground: Rethinking the Human Place in...,William Cronon,Law,http://books.google.com/books/content?id=w04mj...,"Essays by revisionist historians, scientists, ...",1996.0,4.16,560.0,649.0,Uncommon Ground: Rethinking the Human Place in...,9780393315110 Essays by revisionist historians...
2126,9780439569729,0439569729,SCHOLASTIC SUCCESS WITH 4TH GRADE(WORKBOOK),Terry Cooper,Education,http://books.google.com/books/content?id=b-EBH...,"416 bright, colorful pages that give kids prac...",2003.0,4.57,416.0,7.0,SCHOLASTIC SUCCESS WITH 4TH GRADE(WORKBOOK),"9780439569729 416 bright, colorful pages that ..."
2216,9780440421702,0440421705,Hoot,Carl Hiaasen,Bullying,http://books.google.com/books/content?id=uKUTo...,"Roy, who is new to his small Florida community...",2006.0,3.82,292.0,83557.0,Hoot,"9780440421702 Roy, who is new to his small Flo..."
2806,9780571090242,0571090249,Death of a Naturalist,Seamus Heaney,Poetry,http://books.google.com/books/content?id=et6dA...,Death of a Naturalist marks the auspicious out...,1969.0,4.29,57.0,63.0,Death of a Naturalist: Volume one,9780571090242 Death of a Naturalist marks the ...
3000,9780671631987,0671631985,Teach Your Child to Read in 100 Easy Lessons,Siegfried Engelmann;Phyllis Haddox;Elaine Bruner,Education,http://books.google.com/books/content?id=MUucJ...,"With more than half a million copies in print,...",1986.0,4.15,395.0,2083.0,Teach Your Child to Read in 100 Easy Lessons,9780671631987 With more than half a million co...
